In [50]:
import numpy as np # linear algebra
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [51]:
df = pd.read_csv(r'C:\Users\pc\Desktop\Data Science Folder\Retail3\train.csv')
x_pred= pd.read_csv(r'C:\Users\pc\Desktop\Data Science Folder\Retail3\test.csv')


In [52]:
def convert_dates(x):
    x['date']=pd.to_datetime(x['date'])
    x['month']=x['date'].dt.month
    x['year']=x['date'].dt.year
    x['dayofweek']=x['date'].dt.dayofweek
    x.pop('date')
    return x


In [53]:
df = convert_dates(df)
x_pred = convert_dates(x_pred)


In [54]:
def add_avg(x):
    x['daily_avg']=x.groupby(['item','store','dayofweek'])['sales'].transform('mean')
    x['monthly_avg']=x.groupby(['item','store','month'])['sales'].transform('mean')
    return x
df = add_avg(df)

In [55]:
daily_avg = df.groupby(['item','store','dayofweek'])['sales'].mean().reset_index()
monthly_avg = df.groupby(['item','store','month'])['sales'].mean().reset_index()


In [56]:
def merge(x,y,col,col_name):
    x =pd.merge(x, y, how='left', on=None, left_on=col, right_on=col,
            left_index=False, right_index=False, sort=True,
             copy=True, indicator=False,validate=None)
    x=x.rename(columns={'sales':col_name})
    return x
x_pred=merge(x_pred, daily_avg,['item','store','dayofweek'],'daily_avg')
x_pred=merge(x_pred, monthly_avg,['item','store','month'],'monthly_avg')


In [57]:
x_train,x_test,y_train,y_test = train_test_split(df.drop('sales',axis=1),df.pop('sales'),random_state=123,test_size=0.2)


In [58]:
def XGBmodel(x_train,x_test,y_train,y_test):
    matrix_train = xgb.DMatrix(x_train,label=y_train)
    matrix_test = xgb.DMatrix(x_test,label=y_test)
    model=xgb.train(params={'objective':'reg:linear','eval_metric':'mae'}
                    ,dtrain=matrix_train,num_boost_round=500, 
                    early_stopping_rounds=20,evals=[(matrix_test,'test')],)
    return model

model=XGBmodel(x_train,x_test,y_train,y_test)

[14:06:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[0]	test-mae:36.3051
Will train until test-mae hasn't improved in 20 rounds.
[14:06:32] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[1]	test-mae:25.4362
[14:06:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[2]	test-mae:17.8989
[14:06:33] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[3]	test-mae:12.8191
[14:06:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[4]	test-mae:9.59585
[14:06:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[5]	test-mae:7.71342
[14:06:35] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[6]	test-mae:6.6857
[14:06:

[14:06:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[61]	test-mae:5.58234
[14:06:59] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[62]	test-mae:5.58236
[14:07:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 118 extra nodes, 0 pruned nodes, max_depth=6
[63]	test-mae:5.58237
[14:07:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 114 extra nodes, 0 pruned nodes, max_depth=6
[64]	test-mae:5.58251
[14:07:00] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 122 extra nodes, 0 pruned nodes, max_depth=6
[65]	test-mae:5.58263
[14:07:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 106 extra nodes, 0 pruned nodes, max_depth=6
[66]	test-mae:5.58264
[14:07:01] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[67]	test-mae:5.58254
[14:07:02] src/tree/updater_prune.cc:74: tree pruning e

In [59]:
from sklearn.metrics import mean_squared_error


pred_train = model.predict(xgb.DMatrix(x_train), ntree_limit = model.best_ntree_limit)
pred_test = model.predict(xgb.DMatrix(x_test), ntree_limit = model.best_ntree_limit)

rmse = np.sqrt(mean_squared_error(pred_train, y_train))
print('rmse of train : ' , rmse)
               
rmse = np.sqrt(mean_squared_error(pred_test, y_test))
print('rmse of validation : ' , rmse)

rmse of train :  7.179713620675095
rmse of validation :  7.26078515814085


In [65]:
print( model.score(x_train, y_train) )
print( model.score(x_test, y_test))

AttributeError: 'Booster' object has no attribute 'score'

In [60]:
x_pred

,id,store,item,month,year,dayofweek,daily_avg,monthly_avg
0,0,1,1,1,2018,0,15.584615,13.709677
1,7,1,1,1,2018,0,15.584615,13.709677
2,14,1,1,1,2018,0,15.584615,13.709677
3,21,1,1,1,2018,0,15.584615,13.709677
4,28,1,1,1,2018,0,15.584615,13.709677
5,1,1,1,1,2018,1,18.168582,13.709677
6,8,1,1,1,2018,1,18.168582,13.709677
7,15,1,1,1,2018,1,18.168582,13.709677
8,22,1,1,1,2018,1,18.168582,13.709677
9,29,1,1,1,2018,1,18.168582,13.709677


In [61]:
submission = pd.DataFrame(x_pred.pop('id'))

submission

,id
0,0
1,7
2,14
3,21
4,28
5,1
6,8
7,15
8,22
9,29


In [62]:
x_pred

,store,item,month,year,dayofweek,daily_avg,monthly_avg
0,1,1,1,2018,0,15.584615,13.709677
1,1,1,1,2018,0,15.584615,13.709677
2,1,1,1,2018,0,15.584615,13.709677
3,1,1,1,2018,0,15.584615,13.709677
4,1,1,1,2018,0,15.584615,13.709677
5,1,1,1,2018,1,18.168582,13.709677
6,1,1,1,2018,1,18.168582,13.709677
7,1,1,1,2018,1,18.168582,13.709677
8,1,1,1,2018,1,18.168582,13.709677
9,1,1,1,2018,1,18.168582,13.709677


In [63]:
y_pred = model.predict(xgb.DMatrix(x_pred), ntree_limit = model.best_ntree_limit)

y_pred

array([11.905192, 11.905192, 11.905192, ..., 88.389984, 88.389984,
       88.389984], dtype=float32)

In [64]:


submission['sales']= y_pred

submission


,id,sales
0,0,11.905192
1,7,11.905192
2,14,11.905192
3,21,11.905192
4,28,11.905192
5,1,14.773121
6,8,14.773121
7,15,14.773121
8,22,14.773121
9,29,14.773121
